In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
movies =pd.read_csv('./tmdb_movies_data.csv')

In [ ]:
movies.info()

In [ ]:
movies.head()

In [ ]:
movies.columns

In [ ]:
movies = movies.drop(labels={"imdb_id", "popularity", 'budget', 'revenue', 'cast', 'homepage', 'director', 'tagline', 'keywords',
                             'runtime', 'production_companies', 'release_date',
                             'vote_count', 'vote_average', 'release_year', 'budget_adj',
                             'revenue_adj'}, axis=1)

In [ ]:
movies.columns

In [ ]:
movies['tags'] =movies['overview'] + movies['genres']

In [ ]:
movies.columns

In [ ]:
movies = movies.drop(labels={ 'overview', 'genres'}, axis=1)

In [ ]:
movies.columns


In [ ]:
movies.info()

text cleaning

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

In [ ]:
def clean_text(text):
    if not isinstance(text,str):
        return ""
    
    text = text.lower()
    
    text = re.sub(r'[^\w\s\d]','',text)
    
    words =word_tokenize(text)
    
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    
    # join the words back together
    
    text = ''.join(words)
    return text

In [ ]:
movies['tags_clean'] = movies['tags'].apply(clean_text)

In [ ]:
movies.info()

In [ ]:
movies.head()

Count vectorize

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
train_data, test_data = train_test_split(movies, test_size=0.2,random_state=1111)

In [ ]:
cv = CountVectorizer(max_features=10000,stop_words='english')

In [ ]:
vector = cv.fit_transform(movies['tags_clean'].values.astype('U')).toarray()

In [ ]:
vector.shape

In [ ]:
similarity = cosine_similarity(vector)

In [ ]:
similarity

In [ ]:
distance = sorted(list(enumerate(similarity[4])), reverse=True, key= lambda vector:vector[1])
for i in distance[0:5]:
    print(movies.iloc[i[0]].original_title)

In [ ]:
def recommend(movie):
    try:
        index = movies[movies['original_title'] == movie].index[0]
        distance = sorted(list(enumerate(similarity[index])), reverse=True, key= lambda vector:vector[1])
        for i in distance[0:5]:
            print(movies.iloc[i[0]].original_title)
    except:
        print("No movie.")
        

In [ ]:
recommend('Furious 7')

In [ ]:
import pickle

In [ ]:
movies.head(5)

In [ ]:
pickle.dump(movies,open('movies_list.pkl', 'wb'))

In [ ]:
pickle.dump(movies,open('similarity.pkl', 'wb'))

In [ ]:
pickle.load(open('movies_list.pkl', 'rb'))